In [1]:
list1 = list(range(1,4))

In [2]:
list2 = list(range(7,10))

In [6]:
list1

[1, 2, 3]

In [7]:
list2

[7, 8, 9]

In [8]:
for i in range(len(list1)):
    b = list1[i]*list2[i]
    print(b)

7
16
27


In [9]:
c = list()

In [20]:
c = (lambda x : list1*list2)

In [22]:
[lambda x : list1*list2]

[<function __main__.<lambda>(x)>]

In [23]:
import findspark
findspark.init()
findspark.find()

'C:\\Spark'

In [25]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("App").getOrCreate()

In [26]:
def find_create_spark():
    '''Function to locate and create an spark session'''
    import findspark
    findspark.init()
    findspark.find()
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.appName("App").getOrCreate()
    return spark

In [30]:
import requests as re

In [31]:
url = "https://s3.us-east-2.amazonaws.com/assignment1.10/US_category_id.json"

In [33]:
response = re.get(url)
file = response.content

In [36]:
from pyspark import SparkFiles
spark.sparkContext.addFile(url)

In [39]:
df_json = spark.read.option("multiline", "true").json(SparkFiles.get("US_category_id.json"))

In [42]:
df_json.printSchema()

root
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- assignable: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |-- kind: string (nullable = true)



In [46]:
def load_files():
    '''Geting Data and Loading it to DataFrame'''
    from pyspark import SparkFiles
    url_json = "https://s3.us-east-2.amazonaws.com/assignment1.10/US_category_id.json"
    url_csv = "https://s3.us-east-2.amazonaws.com/assignment1.10/USvideos.csv"
    spark.sparkContext.addFile(url_json)
    spark.sparkContext.addFile(url_csv)
    # Making DataFrames
    df_csv = spark.read.csv(SparkFiles.get("USvideos.csv"), header=True, inferSchema=True)
    df_json = spark.read.option("multiline", "true").json(SparkFiles.get("US_category_id.json"))
    return df_csv, df_json
    

In [47]:
load_files()

(DataFrame[video_id: string, trending_date: string, title: string, channel_title: string, category_id: string, publish_time: string, tags: string, views: string, likes: string, dislikes: string, comment_count: string, thumbnail_link: string, comments_disabled: string, ratings_disabled: string, video_error_or_removed: string, description: string],
 DataFrame[etag: string, items: array<struct<etag:string,id:string,kind:string,snippet:struct<assignable:boolean,channelId:string,title:string>>>, kind: string])

In [52]:
def preprocessing_data():
    '''Cleaning The DataFrame and Assiging Proper inferSchema to them i.e (DataTypes)'''
    #Processing CSV:
    from pysaprk.sql.types import IntegerType  ## Imporing DataTypes
    num_list = list(['category_id', 'views', 'likes', 'dislikes', 'comment_count'])
    for num in num_list:
        df_csv = df_csv.withColumn(num,  col= df_csv[num].cast(IntegerType())) ## Defining DataType
    
    
    #Processing JSON:
    df_json2 = df_json.withColumn('new', arrays_zip("items.kind", 'items.etag', 'items.id', 'items.snippet.assignable','items.snippet.channelID', 'items.snippet.title'))\
    .withColumn('new', explode("new")) ## Selecting Data and Exploding it
    
    df_json3 = df_json2.select('new.*') ## Creating New DataFrame for the Data pipeline
    
    #Renaming Exploded Columns:
    df_json3 = df_json3.withColumnRenamed('0', 'kind')
    df_json3 = df_json3.withColumnRenamed('1','etag')                                              #Renaming Our Columns
    df_json3 = df_json3.withColumnRenamed('2','id')
    df_json3 = df_json3.withColumnRenamed('3', 'assignable')
    df_json3 = df_json3.withColumnRenamed('4', 'ChannelID')
    df_json3 = df_json3.withColumnRenamed('5', 'cat_title')
    
    
    # Joining the two Dataframe on category_id:
    
    df_join = df_csv.join(df_json3, df_csv['category_id'] == df_json3['id'])

In [53]:
def filtering():
    df_is_nlf = df_join.filter(df_join.channel_title.contains('NFL'))
    df_not_nlf = df_join.filter(~df_join.channel_title.contains('NFL'))
    

In [60]:
df = spark.read.csv('../LMT/Quiz_4/dataR2.csv')

In [70]:
df.write.csv("../airflow_docker/store_files_redshift/new")


AnalysisException: path file:/C:/Users/akash/JupyterWorkbook/airflow_docker/store_files_redshift/new already exists.

In [71]:
import numpy as np
tdarray = np.array([[1, 7, 5],[2, 7, 6],[3, 6, 8],[4, 2, 6],[5, 99, 25],[6, 3 , 9],[7, 0, 1],[8, 0, 0],[9, 0, 0]])

In [75]:
type(tdarray)

numpy.ndarray

In [76]:
tdarray

array([[ 1,  7,  5],
       [ 2,  7,  6],
       [ 3,  6,  8],
       [ 4,  2,  6],
       [ 5, 99, 25],
       [ 6,  3,  9],
       [ 7,  0,  1],
       [ 8,  0,  0],
       [ 9,  0,  0]])

In [82]:
np.array([[tdarray[4][1],tdarray[4][2]], [tdarray[5][1],tdarray[5][2]], [tdarray[6][1],tdarray[6][2]]])

array([[99, 25],
       [ 3,  9],
       [ 0,  1]])